# Advanced tour of the Bayesian Optimization package

In [1]:
import sys
sys.path.append('../')
from bayes_opt import BayesianOptimization, DiscreteBayesianOptimization
from bayes_opt.event import Events, DEFAULT_EVENTS
import time
import numpy as np
n_threads=8
from bayes_opt import UtilityFunction

In [2]:
from test_functions import Parabolic

## An easy trial function

In [3]:
# Let's start by definying our function, bounds, and instanciating an optimization object.
def black_box_function(x, y):
    return -x ** 2 - (y - 1) ** 2 + 1

In [4]:
optimizer = DiscreteBayesianOptimization(
    f=None,
    prange={'x': [-2, 2,.3], 'y': [-3, 3,.3]},
    verbose=2,
    random_state=1
)

In [5]:
utility = UtilityFunction(kind="ucb", kappa=2.5, xi=0.0)

In [6]:
next_point_to_probe = optimizer.suggest(utility)
print("Next point to probe is:", next_point_to_probe)
target = black_box_function(**next_point_to_probe[0])
print("Found the target value to be:", target)

Next point to probe is: [{'x': -0.5, 'y': 1.2000000000000002}]
Found the target value to be: 0.71


In [7]:
optimizer.register(
    params=next_point_to_probe[0],
    target=target,
)

## Vladimir's test function

In [8]:
DIM=4
prange={}
pbounds={}
for i in range(DIM):
    prange[f'x_{i}'] = (0,1,0.01)
    pbounds[f'x_{i}'] = (0,1)
parabolic = Parabolic('joint',DIM)
func = parabolic.f
print(prange)

{'x_0': (0, 1, 0.01), 'x_1': (0, 1, 0.01), 'x_2': (0, 1, 0.01), 'x_3': (0, 1, 0.01)}


## Discrete optimization using  self-contained optimizer

In [9]:
start = time.time()
n_iter =2
dbo = DiscreteBayesianOptimization(
    f=parabolic.f,
    prange=prange,
    verbose=2,
    random_state=1
)
dbo.maximize(init_points=2,n_iter=n_iter)
print("Maximum value found using discrete GPs: {}".format(dbo.max['target']))
print("Time taken: {} seconds for {} iterations".format(time.time()-start,n_iter))
del dbo

|   iter    |  target   |    x_0    |    x_1    |    x_2    |    x_3    |
-------------------------------------------------------------------------
|  1        | -1.567    |  0.417    |  0.7203   |  0.000114 |  0.3023   |
|  1        | -1.567    |  0.41     |  0.72     |  0.0      |  0.3      |
|  2        | -1.866    |  0.1468   |  0.09234  |  0.1863   |  0.3456   |
|  2        | -1.866    |  0.14     |  0.09     |  0.18     |  0.34     |
|  3        | -0.04125  |  1.0      |  1.0      |  1.0      |  1.0      |
|  3        | -0.04125  |  1.0      |  1.0      |  1.0      |  1.0      |
|  4        |  0.9864   |  1.0      |  1.0      |  1.0      |  0.0      |
|  4        |  0.9864   |  1.0      |  1.0      |  1.0      |  0.0      |
Maximum value found using discrete GPs: 0.986418244015935
Time taken: 1.4327080249786377 seconds for 2 iterations


## Continuous optimization using self-contained optimizer

In [10]:
start = time.time()
n_iter =2
bo = BayesianOptimization(
    f=parabolic.f,
    pbounds=pbounds,
    verbose=2,
    random_state=1
)
bo.maximize(init_points=2,n_iter=n_iter)
print("Maximum value found using discrete GPs: {}".format(bo.max['target']))
print("Time taken: {} seconds for {} iterations".format(time.time()-start,n_iter))

|   iter    |  target   |    x_0    |    x_1    |    x_2    |    x_3    |
-------------------------------------------------------------------------
|  1        | -1.567    |  0.417    |  0.7203   |  0.000114 |  0.3023   |
|  2        | -1.866    |  0.1468   |  0.09234  |  0.1863   |  0.3456   |
|  3        | -0.04125  |  1.0      |  1.0      |  1.0      |  1.0      |
|  4        |  0.9864   |  1.0      |  1.0      |  1.0      |  0.0      |
Maximum value found using discrete GPs: 0.986418244015935
Time taken: 1.371074914932251 seconds for 2 iterations


## Discrete optimization using class methods
This will be how the general KUKA program is written. 

In [11]:
start = time.time()
n_iter =4
dbo = DiscreteBayesianOptimization(
    f=None,
    prange=prange,
    verbose=2,
    random_state=1
)
dbo._prime_subscriptions()
dbo.dispatch(Events.OPTMIZATION_START)
for _ in range(n_iter):
    next_points = dbo.suggest(utility,n_acqs=3,n_iter=250,multiprocessing=4)
    target = []
    for next_point in next_points:
        for key in next_point:
            next_point[key] += np.random.uniform(-.001,.001) #makes discrete point diff than bin
        target = parabolic.f(**next_point)
        dbo.register(params=next_point, target=target)
    dbo.dispatch(Events.BATCH_END)
dbo.dispatch(Events.OPTMIZATION_END)
print("Maximum value found using discrete GPs: {}".format(dbo.max['target']))
print("Time taken: {} seconds for {} iterations".format(time.time()-start,n_iter))

|   iter    |  target   |    x_0    |    x_1    |    x_2    |    x_3    |
-------------------------------------------------------------------------
|  1        | -1.583    |  0.4101   |  0.7193   | -0.000185 |  0.3005   |
|  1        | -1.583    |  0.41     |  0.71     | -0.01     |  0.3      |
|  2        | -1.914    |  0.139    |  0.09092  |  0.1803   |  0.3404   |
|  2        | -1.914    |  0.13     |  0.09     |  0.18     |  0.34     |
|  3        | -0.1384   |  0.3901   |  0.5294   |  0.4093   |  0.6805   |
|  3        | -0.1384   |  0.39     |  0.52     |  0.4      |  0.68     |
END BATCH----------------------------------------------------------------
|  4        | -0.164    |  0.4193   |  0.5098   |  0.3898   |  0.6991   |
|  4        | -0.164    |  0.41     |  0.5      |  0.38     |  0.69     |
|  5        | -0.1134   |  0.3701   |  0.4904   |  0.4291   |  0.6699   |
|  5        | -0.1134   |  0.37     |  0.49     |  0.42     |  0.66     |
|  6        | -0.06711  |  0.4404   | 

## Discrete optimization using class methods
Demonstrating slice sampling.

In [12]:
start = time.time()
n_iter =4
dbo = DiscreteBayesianOptimization(
    f=None,
    prange=prange,
    verbose=2,
    random_state=1
)
dbo._prime_subscriptions()
dbo.dispatch(Events.OPTMIZATION_START)
for _ in range(n_iter):
    next_points = dbo.suggest(utility,sampler='KMBBO', n_acqs=3,n_iter=250,multiprocessing=4)
    target = []
    for next_point in next_points:
        for key in next_point:
            next_point[key] += np.random.uniform(-.001,.001) #makes discrete point diff than bin
        target = parabolic.f(**next_point)
        dbo.register(params=next_point, target=target)
    dbo.dispatch(Events.BATCH_END)
dbo.dispatch(Events.OPTMIZATION_END)
print("Maximum value found using discrete GPs: {}".format(dbo.max['target']))
print("Time taken: {} seconds for {} iterations".format(time.time()-start,n_iter))

|   iter    |  target   |    x_0    |    x_1    |    x_2    |    x_3    |
-------------------------------------------------------------------------
|  1        | -1.538    |  0.4098   |  0.7209   |  0.000537 |  0.3009   |
|  1        | -1.538    |  0.4      |  0.72     |  0.0      |  0.3      |
|  2        | -1.568    |  0.1397   |  0.09009  |  0.1803   |  0.3395   |
|  2        | -1.568    |  0.13     |  0.09     |  0.18     |  0.33     |
|  3        |  0.5683   |  0.3909   |  0.5291   |  0.4092   |  0.6808   |
|  3        |  0.5683   |  0.39     |  0.52     |  0.4      |  0.68     |
END BATCH----------------------------------------------------------------
|  4        |  0.4268   |  0.361    |  0.2394   |  0.3905   |  0.6998   |
|  4        |  0.4268   |  0.36     |  0.23     |  0.39     |  0.69     |
|  5        |  1.158    |  0.61     |  0.7693   |  0.5703   |  0.6293   |
|  5        |  1.158    |  0.6      |  0.76     |  0.57     |  0.62     |
|  6        |  0.3423   |  0.53     | 

## Discrete constraiend optimization using class methods

In [13]:
start = time.time()
n_iter =10
constraints = ['2.5 - x_0 - x_1 - x_2']
dbo = DiscreteBayesianOptimization(
    f=None,
    prange=prange,
    verbose=2,
    random_state=1,
    constraints=constraints
)
dbo._prime_subscriptions()
dbo.dispatch(Events.OPTMIZATION_START)
for _ in range(n_iter):
    next_points = dbo.suggest(utility,n_acqs=3,n_iter=250)
    target = []
    for next_point in next_points:
        for key in next_point:
            next_point[key] += np.random.uniform(-.001,.001) #makes discrete point diff than bin
        target = parabolic.f(**next_point)
        dbo.register(params=next_point, target=target)
    dbo.dispatch(Events.BATCH_END)
dbo.dispatch(Events.OPTMIZATION_END)
print("Maximum value found using discrete GPs: {}".format(dbo.max['target']))
print("Time taken: {} seconds for {} iterations".format(time.time()-start,n_iter))

|   iter    |  target   |    x_0    |    x_1    |    x_2    |    x_3    |
-------------------------------------------------------------------------
|  1        | -1.543    |  0.4106   |  0.7199   | -0.000943 |  0.2994   |
|  1        | -1.543    |  0.41     |  0.71     | -0.01     |  0.29     |
|  2        | -1.571    |  0.1403   |  0.08916  |  0.1792   |  0.3391   |
|  2        | -1.571    |  0.14     |  0.08     |  0.17     |  0.33     |
|  3        |  0.5708   |  0.3905   |  0.5291   |  0.4108   |  0.6795   |
|  3        |  0.5708   |  0.39     |  0.52     |  0.41     |  0.67     |
END BATCH----------------------------------------------------------------
|  4        |  3.096    |  0.981    |  0.3392   |  0.84     |  0.8909   |
|  4        |  3.096    |  0.98     |  0.33     |  0.83     |  0.89     |
|  5        | -0.1792   |  0.9896   |  0.7698   |  0.2005   |  0.3304   |
|  5        | -0.1792   |  0.98     |  0.76     |  0.2      |  0.33     |
|  6        |  1.843    |  0.989    | 

In [14]:
start = time.time()
n_iter =10
constraints = ['3 - x_0 - x_1 - x_2']
dbo = DiscreteBayesianOptimization(
    f=None,
    prange=prange,
    verbose=2,
    random_state=1,
    constraints=constraints
)
dbo._prime_subscriptions()
dbo.dispatch(Events.OPTMIZATION_START)
for _ in range(n_iter):
    next_points = dbo.suggest(utility,sampler='KMBBO', n_acqs=3,n_iter=250,multiprocessing=4)

    target = []
    for next_point in next_points:
        for key in next_point:
            next_point[key] += np.random.uniform(-.001,.001) #makes discrete point diff than bin
        target = parabolic.f(**next_point)
        dbo.register(params=next_point, target=target)
    dbo.dispatch(Events.BATCH_END)
dbo.dispatch(Events.OPTMIZATION_END)
print("Maximum value found using discrete GPs: {}".format(dbo.max['target']))
print("Time taken: {} seconds for {} iterations".format(time.time()-start,n_iter))

|   iter    |  target   |    x_0    |    x_1    |    x_2    |    x_3    |
-------------------------------------------------------------------------
|  1        | -1.54     |  0.4098   |  0.7191   | -0.000103 |  0.2993   |
|  1        | -1.54     |  0.4      |  0.71     | -0.01     |  0.29     |
|  2        | -1.566    |  0.14     |  0.09059  |  0.1802   |  0.3403   |
|  2        | -1.566    |  0.13     |  0.09     |  0.18     |  0.34     |
|  3        |  0.5682   |  0.3904   |  0.5307   |  0.4107   |  0.6797   |
|  3        |  0.5682   |  0.39     |  0.53     |  0.41     |  0.67     |
END BATCH----------------------------------------------------------------
|  4        |  0.4268   |  0.3609   |  0.2392   |  0.3906   |  0.6998   |
|  4        |  0.4268   |  0.36     |  0.23     |  0.39     |  0.69     |
|  5        |  1.155    |  0.6091   |  0.7696   |  0.57     |  0.6304   |
|  5        |  1.155    |  0.6      |  0.76     |  0.57     |  0.63     |
|  6        |  0.3498   |  0.5301   | 